In [1]:
import genanki
import csv
import argparse
import os
import hashlib
import time
from pathlib import Path
import logging

from src.anki import read_file, make_deck, make_model, make_note, make_package, read_package
from src.media import add_images, add_phonetics
from src.log_config import configure_logging

configure_logging(ignore_libs=["PIL", "phonemizer"])

INPUT_DIR = Path("input/")
OUTPUT_DIR = Path("output/")
TEMPLATE_DIR = Path("template/")
MEDIA_DIR = Path("media/")
IMG_DIR = MEDIA_DIR / "img"
AUDIO_DIR = MEDIA_DIR / "sound"
for d in [INPUT_DIR, OUTPUT_DIR, TEMPLATE_DIR, MEDIA_DIR, IMG_DIR, AUDIO_DIR]:
    d.mkdir(parents=True, exist_ok=True)

file_name = "Cours_01_QA"
file_type = ".md"
df_file = INPUT_DIR / (file_name + file_type)
apkg_file = OUTPUT_DIR / (file_name + ".apkg")
style = TEMPLATE_DIR / "style.css"


In [2]:
df = read_file(df_file)
df = add_phonetics(df)
df, media_files = add_images(df, IMG_DIR, force_replace=False)
deck = make_deck(df)
model = make_model(df, style)
successful_cards, total_rows = make_note(deck, model, df)
make_package(deck, media_files=media_files, apkg_file=apkg_file)
logging.info(f"Created {successful_cards} cards out of {total_rows} possible lines.")
df.head()

[19:28:20] [INFO] [media] - Added phonetics for 4 words.
[19:28:20] [INFO] [root] - Created 4 cards out of 4 possible lines.


,Front,Back,Extra,More,Phonetics,Audio
0,"Qu'est-ce que ça veut dire ""l’amour"" en allema...","Ça veut dire ""die Liebe"" en allemand.",(= Was bedeutet *...* auf Deutsch?),,"/kɛskˌə sa vˈø dˈiʁ ""lamˈuʁ"" ɑ̃n almˈɑ̃ ?/",
1,"Comment on dit ""der Mitbewohner"" en français ?","On dit ""le colocataire"" en français.",(= Wie sagt man *...* auf Französisch?),,"/kɔmˌɑ̃ ɔ̃ dˈi ""dˈɛʁ (en)mˈɪtbjuːˌəʊnə(fr)"" ɑ̃...",
2,Comment ça s'écrit ?,Ça s'écrit D-E-U-X,(= Wie schreibt man *...* ?),,/kɔmˌɑ̃ sa sekʁˈi ?/,
3,Comment ça se prononce ?,Ça se prononce …,= Wie spricht man *...* aus?),,/kɔmˌɑ̃ sa sə- pʁonˈɔ̃s ?/,


In [3]:
def main():
    parser = argparse.ArgumentParser(description='Create Anki decks from CSV or simple Markdown files.')
    parser.add_argument('file', help='File to import (CSV or Markdown). The file must have a header row that defines the fields for the Anki cards.')
    parser.add_argument('--deck-name', default=None, help='Optional: Custom name for the Anki deck. If not provided, the deck name is derived from the file name.')
    parser.add_argument('--package-name', default=None, help='Optional: Custom name for the Anki package file (without .apkg extension). If not provided, the package name is the same as the deck name.')

    args = parser.parse_args()

    file_name = args.file
    deck_name = args.deck_name if args.deck_name else os.path.splitext(os.path.basename(file_name))[0]
    package_name = args.package_name if args.package_name else deck_name + '.apkg'

    if not os.path.exists(file_name):
        print(f"Error: The file {file_name} does not exist.")
        return

    try:
        file_data = read_file(file_name)
    except ValueError as e:
        print(e)
        return
    

    